In [11]:
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
from alpaca_trade_api.rest import REST, TimeFrame
import os 
import pandas as pd
from time import sleep 


#!pip install python-dotenv
alpaca_api_key = os.getenv("Alpaca_Paper_Key_ID")
alpaca_secret_key = os.getenv("Alpaca_Paper_Secret_Key")
load_dotenv("key.env")
alpaca = REST(alpaca_api_key, alpaca_secret_key, "https://paper-api.alpaca.markets")

In [12]:
def alpaca_caller(ticker, start_date, end_date):
    
    """ esta herramienta te permite accedar a la data de los tikers, los parametros son nombre del ticker, la fecha de inicio, y l afecha de finalisado, todo con comillas"""
    
    alpaca_api_key = os.getenv("Alpaca_Paper_Key_ID")
    alpaca_secret_key = os.getenv("Alpaca_Paper_Secret_Key")
    load_dotenv("key.env")
    alpaca = tradeapi.REST(alpaca_api_key, alpaca_secret_key, api_version = "V2"
    )
    

    ticker = ticker
    timeframe = "1D"
    start_date = pd.Timestamp(start_date, tz="America/New_York").isoformat()
    end_date = pd.Timestamp(end_date, tz="America/New_York").isoformat()
    ticker_data = alpaca.get_bars(ticker, TimeFrame.Hour, start_date, end_date, adjustment='raw').df
    return ticker_data


## equal wheight
def _target_equal_weights(ticker_list):

    target_equal_weights = {}

    for ticker in ticker_list:
        target_equal_weights[ticker] = 1 / len(ticker_list)

    return target_equal_weights

target_equal_weights = _target_equal_weights(["FB", "AAPL", "AMZN", "NFLX", "GOOG"])

# {'AAPL': 0.2, 'AMZN': 0.2, 'FB': 0.2, 'GOOG': 0.2, 'NFLX': 0.2}

In [ ]:
## Market cap rebalance

import requests
import numpy as np
import pandas as pd

api_key = "YOUR_INTRINIO_API_KEY_HERE"

def _market_cap(ticker):

    res = requests.get(f"https://api-v2.intrinio.com/companies/{ticker}/historical_data/marketcap?api_key={api_key}")

    if res.status_code == 200:
        historical_data = res.json().get("historical_data")
        market_cap = historical_data[0].get("value", 0)
        return market_cap

    return 0

def _market_caps(ticker_list):

    market_caps = {}
    for ticker in ticker_list:
        market_caps[ticker] = _market_cap(ticker)

    return market_caps

def _target_market_cap_weights(ticker_list):

    target_market_cap_weights = {}
    market_caps = _market_caps(ticker_list)
    market_caps_list = list(market_caps.values())
    market_cap_weights = market_caps_list / np.sum(market_caps_list)

    idx = 0
    while idx < len(ticker_list):
        target_market_cap_weights[ticker_list[idx]] = round(market_cap_weights[idx], 3)
        idx += 1

    return target_market_cap_weights

target_market_cap_weights = _target_market_cap_weights(["FB", "AAPL", "AMZN", "NFLX", "GOOG"])

# {'AAPL': 0.404, 'AMZN': 0.235, 'FB': 0.094, 'GOOG': 0.243, 'NFLX': 0.024}


In [13]:
portfolio_symbols = ["AAPL", "MSFT","AMZN","GOOGL","META","TSLA","V"]

In [14]:
def _all_symbols_eligible_for_fractionals(portfolio_symbols):
    
    all_symbols_eligible_for_fractionals = all([[alpaca.get_asset(symbol).fractionable for symbol in portfolio_symbols]])
    return all_symbols_eligible_for_fractionals
_all_symbols_eligible_for_fractionals(portfolio_symbols)

True

In [32]:
orders = alpaca.list_orders(status='open')
alpaca_positions = alpaca.list_positions()

In [53]:
def alpaca_symbols_to_close(alpaca_positions, portfolio_symbols):
    alpaca_symbols_to_close = [x for x in alpaca_positions if x not in portfolio_symbols]
    return alpaca_symbols_to_close

alpaca_symbols_to_close = alpaca_symbols_to_close(alpaca_positions, portfolio_symbols)
alpaca_symbols_to_close 

[Position({   'asset_class': 'crypto',
     'asset_id': '35f33a69-f5d6-4dc9-b158-4485e5e92e4b',
     'asset_marginable': False,
     'avg_entry_price': '1333.5',
     'change_today': '-0.0076587271195527',
     'cost_basis': '132.9832875',
     'current_price': '1295.7',
     'exchange': 'FTXU',
     'lastday_price': '1305.7',
     'market_value': '129.2136825',
     'qty': '0.099725',
     'qty_available': '0.099725',
     'side': 'long',
     'symbol': 'ETHUSD',
     'unrealized_intraday_pl': '-0.99725',
     'unrealized_intraday_plpc': '-0.0076587271195527',
     'unrealized_pl': '-3.769605',
     'unrealized_plpc': '-0.0283464566929134'})]

In [54]:
def alpaca_close_positions(symbols_to_close):

    for symbol in symbols_to_close:
        order_pending = True
        alpaca_order_info = alpaca.close_position(symbol)
        alpaca_client_order_id = alpaca_order_info.client_order_id
        
        while order_pending:
            alpaca_latest_order_info = alpaca.get_order_by_client_order_id(alpaca_client_order_id)
            alpaca_latest_status = alpaca_latest_order_info.status
            if alpaca_latest_status == "filled":
                order_pending = False


In [69]:

def _rebalance_equity(cash_weight):
    
    alpaca_account_equity = float(alpaca.get_account().equity)
    rebalance_equity = alpaca_account_equity - (alpaca_account_equity * cash_weight)
    
    return rebalance_equity

balance = alpaca.get_account().equity
print(balance)

rebalance_equity = _rebalance_equity(.05)
print(rebalance_equity)

94623.4345075
89892.262782125


In [70]:

def _portfolio_symbols_equity_allocations(target_allocations, rebalance_equity):

    portfolio_symbols_equity_allocations = {}
    for ticker, weight in target_allocations.items():
        portfolio_symbols_equity_allocations[ticker] = round((weight * rebalance_equity), 2)
    
    return portfolio_symbols_equity_allocations

ModuleNotFoundError: No module named 'portfolio_spanol'

In [57]:

def _alpaca_latest_positions_allocations():
    
    latest_alpaca_positions_allocations = {position.symbol: float(position.market_value) for position in alpaca.list_positions()}
    return latest_alpaca_positions_allocations

def _alpaca_latest_positions():
    
    alpaca_positions = alpaca.list_positions()
    alpaca_latest_positions = [position.symbol for position in alpaca_positions]

    return alpaca_latest_positions


In [66]:
print(_alpaca_latest_positions_allocations())
print(_alpaca_latest_positions())

{'ETHUSD': 129.00426}
['ETHUSD']


In [59]:

def _alpaca_symbols_to_sell_and_buy(portfolio_symbols_equity_allocations, latest_alpaca_positions_allocations):

    positions_to_sell, positions_to_buy = {}, {}

    # Loop Through Latest Desired Allocation
    for ticker, desired_allocation in portfolio_symbols_equity_allocations.items():

        current_allocation = latest_alpaca_positions_allocations.get(ticker, 0)
        allocation_to_adjust = desired_allocation - current_allocation

        if allocation_to_adjust > 0:
            positions_to_buy[ticker] = allocation_to_adjust
        else:
            positions_to_sell[ticker] = allocation_to_adjust * -1
    
    return positions_to_sell, positions_to_buy

In [60]:
def alpaca_order(symbol, amount, side):
    
    if amount > 1:
        
        alpaca_order_info = alpaca.submit_order(symbol=symbol, notional=amount, side=side, type="market", time_in_force="day")
        alpaca_client_order_id = alpaca_order_info.client_order_id
        
        order_pending = True
        while order_pending:
            latest_alpaca_order_info = alpaca.get_order_by_client_order_id(alpaca_client_order_id)
            alpaca_latest_status = latest_alpaca_order_info.status
            sleep(2.5)
            if alpaca_latest_status == "filled":
                order_pending = False

def handle_buy_orders(positions_to_buy):
    
    for symbol, amount in positions_to_buy.items():
        alpaca_order(symbol, amount, "buy")

def handle_sell_orders(positions_to_sell):
    
    for symbol, amount in positions_to_sell.items():
        alpaca_order(symbol, amount, "sell")

In [61]:
def alpaca_rebalance(target_allocations, cash_weight=.05):

    portfolio_symbols = list(target_allocations.keys())

    # Check to Make Sure All Symbols are Eligible for Fractional Trading on Alpaca and Market is Open
    all_symbols_eligible_for_fractionals = _all_symbols_eligible_for_fractionals(portfolio_symbols)

    # Ensures All Symbols are Fractionable and the Market is Open
    if all_symbols_eligible_for_fractionals and alpaca.get_clock().is_open:

        # Grab Current Alpaca Holdings
        alpaca_latest_positions = _alpaca_latest_positions()

        # Construct a List of Equities to Close Based on Current Alpaca Holdings and Current Desired Holdings
        print("Closing Positions...")
        print(20*"~~")
        alpaca_symbols_to_close = _alpaca_symbols_to_close(alpaca_latest_positions, portfolio_symbols)
        
        # Close Any Alpaca Positions if Neccessary
        if alpaca_symbols_to_close:
            alpaca_close_positions(alpaca_symbols_to_close)

        # Calculate Rebalance Weight Taking Cash Weight % into Account
        print("Preparing Rebalance Equity...")
        print(20*"~~")
        rebalance_equity = _rebalance_equity(cash_weight)
        
        # Allocate the Equity to Each Holding Based on Weight and Available Portfolio Equity
        print("Preparing Positions to Sell and Buy...")
        print(20*"~~")
        portfolio_symbols_equity_allocations = _portfolio_symbols_equity_allocations(target_allocations, rebalance_equity)
        latest_alpaca_positions_allocations = _alpaca_latest_positions_allocations()
        positions_to_sell, positions_to_buy = _alpaca_symbols_to_sell_and_buy(portfolio_symbols_equity_allocations, latest_alpaca_positions_allocations)

        # Finally Adjust Allocations 
        print("Rebalancing...")
        print(20*"~~")
        handle_sell_orders(positions_to_sell)
        handle_buy_orders(positions_to_buy)

        print("Completed Rebalance!")
        print(20*"~~")

target_allocations = {"VOO": .2, "VGT": .2, "QQQ": .2, "VTI": .2, "VYM": .2}

alpaca_rebalance(target_allocations)


TypeError: unsupported operand type(s) for /: 'set' and 'int'